In [4]:
import numpy as np
import pandas as pd
import networkx as nx
import statistics
from models.dynotears_modified import from_pandas_dynamic


In [5]:
def calculate_anomaly(data, node_path):
    # 读取原始矩阵 (20+18)*2
    # data = np.load(data_path)
    print("data shape:", data.shape)
    
    # 读取节点名称
    node_name = pd.read_csv(node_path, index_col=0)["0"].tolist()
    print("node num:", len(node_name))

    # 提取需要测评的矩阵-即左半边矩阵
    data_extract = data[:76, :38]
    print("half data shape:", data_extract.shape)

    # 转化为DataFrame
    df_data_extract = pd.DataFrame(data_extract, index=node_name+node_name, columns=node_name)
    # print(df_data_extract.head())

    # 提取出异常测评DataFrame-即字段名中出现'冷通道温度'的行，与字段名中不出现'冷通道温度'的列
    df_rows_extract = df_data_extract.loc[df_data_extract.index.str.contains("冷通道温度")]    
    df_rows_cols_extract = df_rows_extract.loc[:, ~df_rows_extract.columns.str.contains("冷通道温度")]
    print("extract data shape:", df_rows_cols_extract.shape)
    # df_rows_cols_extract.head()

    # 计算不符合实际的数目和 
    # 直接对dataframe求和即为（冷通道温度，送风温度）值为1的总数。
    wrong_num = int(df_rows_cols_extract.sum().sum())
    print("wrong num:", wrong_num)

    # 找出值为1的位置
    pos = df_rows_cols_extract.where(df_rows_cols_extract == 1).stack().reset_index()
    print(pos)
    
    return wrong_num

In [6]:
load_path = '/home/lipeiwen.lpw/TECDI/data_real/downstream/'
num_nodes = 38
num_dataset = 10
all_wrong = np.zeros(num_dataset)
node_path = "/home/lipeiwen.lpw/TECDI/data_real/realdata_preprocess/transfered/downstream_less/node_name.csv"
for i_dataset in range(1,num_dataset+1):
    df_g = pd.DataFrame(np.load(load_path+f'data{i_dataset}.npy')).iloc[:,:num_nodes]
    df_g = df_g.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
    g_learnt, w_est, a_est  = from_pandas_dynamic(df_g, 1, lambda_w=.001,lambda_a=.001,w_threshold=.5) #0.25
    w_est = np.where(w_est==0,w_est,1)
    a_est = np.where(a_est==0,a_est,1)
    dag = np.hstack((np.vstack((w_est,a_est)),np.zeros((w_est.shape[0]+a_est.shape[0],a_est.shape[0]))))
    np.save(f'/home/lipeiwen.lpw/TECDI/baseline/DYNOTEARS/downstream/DAG{i_dataset}.npy',dag)
    print(np.count_nonzero(dag==1))
    wrong_num = calculate_anomaly(dag, node_path)
    all_wrong[i_dataset-1] = wrong_num
        # print(wrong_num)
print('wrong mean:',np.mean(all_wrong),'\n','wrong stdev:',statistics.stdev(all_wrong))

38
data shape: (76, 76)
node num: 38
half data shape: (76, 38)
extract data shape: (36, 20)
wrong num: 0
Empty DataFrame
Columns: [level_0, level_1, 0]
Index: []
38
data shape: (76, 76)
node num: 38
half data shape: (76, 38)
extract data shape: (36, 20)
wrong num: 0
Empty DataFrame
Columns: [level_0, level_1, 0]
Index: []
38
data shape: (76, 76)
node num: 38
half data shape: (76, 38)
extract data shape: (36, 20)
wrong num: 0
Empty DataFrame
Columns: [level_0, level_1, 0]
Index: []
38
data shape: (76, 76)
node num: 38
half data shape: (76, 38)
extract data shape: (36, 20)
wrong num: 0
Empty DataFrame
Columns: [level_0, level_1, 0]
Index: []
38
data shape: (76, 76)
node num: 38
half data shape: (76, 38)
extract data shape: (36, 20)
wrong num: 0
Empty DataFrame
Columns: [level_0, level_1, 0]
Index: []
38
data shape: (76, 76)
node num: 38
half data shape: (76, 38)
extract data shape: (36, 20)
wrong num: 0
Empty DataFrame
Columns: [level_0, level_1, 0]
Index: []
38
data shape: (76, 76)
node

In [7]:

# print('edge mean:',np.mean(edge),'\n','edge stdev:',np.std(all_wrong))